In [2]:
from utils import *
import gensim
from sklearn.mixture import BayesianGaussianMixture
import json

In [3]:
from utils import *

import gensim
from sklearn.mixture import BayesianGaussianMixture
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from semiSupervisedDnn import selfTrainer

In [4]:
df = pd.read_csv("assets/finalproduct/finalproductDf")
df.drop(["Unnamed: 0"],axis=1, inplace=True)
id_to_auth = pickle_o.load("assets/dictionaries/id_to_all_auths_2004")
auth_to_id = pickle_o.load("assets/dictionaries/auths_to_all_id_2004")

In [5]:
Name = list(df.Author.values)
kth_id = [auth_to_id[a] for a in Name]
df_only_auth = pd.DataFrame(data={"Name":Name, "ID":kth_id})
df_only_auth.to_csv("assets/finalproduct/onlyAuthors.csv")

In [6]:
df_abs = pd.read_csv("assets/dataframes/all_authors_df_2004")
df_abs.drop(["Unnamed: 0"],axis=1, inplace=True)

In [7]:
all_auth = pd.read_csv("assets/dataframes/KTH_UPPSALA_SODERTORN_SU")
all_auth.drop(["Unnamed: 0"],axis=1, inplace=True)

In [8]:
kth_id = list()
for d in all_auth.Doc_id:
    try:
        kth_id.append(int(d))
    except:
        pass

In [9]:
len(kth_id)

23811

In [10]:
ls_d = list()
for d in df.Doc_id.values:
    print(ls_d)
    ls_d += d.split(":")
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
kth_id

[65923903,
 40520967,
 41290309,
 40129874,
 61385618,
 336,
 40924748,
 478,
 73796495,
 60732308,
 58541795,
 40236185,
 40368445,
 58885734,
 74275918,
 74275767,
 72258099,
 40541502,
 41178885,
 41086202,
 41744330,
 78381912,
 40561648,
 40820121,
 39968045,
 64672598,
 40825552,
 41226858,
 67261454,
 40755327,
 59081874,
 40490716,
 40999516,
 59137733,
 40686336,
 65948650,
 41714575,
 40119425,
 40518690,
 42131810,
 40934677,
 41628706,
 41757102,
 58718488,
 41456719,
 40582303,
 42010627,
 42010626,
 40681927,
 41654954,
 40525925,
 59706886,
 81616148,
 40307681,
 1203,
 78158455,
 67093343,
 40166773,
 40809094,
 41575863,
 58553432,
 40780688,
 58608322,
 58869849,
 58796200,
 41663171,
 40520086,
 58938541,
 41547530,
 40882550,
 41023772,
 41150101,
 41668656,
 78409388,
 41533653,
 41879559,
 40659935,
 40447032,
 41260425,
 40130320,
 65853123,
 60791275,
 63895790,
 40506237,
 40691323,
 60878860,
 58971365,
 40397780,
 68540949,
 59710265,
 40595163,
 76830526,
 7

In [29]:
train_label = list()
for d in kth_id:
    if str(d) in ls_d:
        train_label.append(0)
    else:
        train_label.append(1)

In [30]:
kth_vec = model[np.array(kth_id).astype(str)]
kth_label = np.asarray(train_label)

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(kth_vec, kth_label, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [35]:
sum(y_val)

2067

In [36]:
len(y_val)

2381

In [ ]:
sum(y_test == 1) / len(y_test)

In [37]:
b = GaussianNB()
b.fit(X_train, y_train)

test_pred = b.predict(X_test)
print(accuracy_score(test_pred, y_test))

0.9298908480268682


In [38]:
article_tag = [t for t in doc_tag if not t[0].islower()]

In [39]:
unlabeled_tags = list(set(article_tag) - set(np.array(kth_id).astype(str)))

In [40]:
unlabeled_vec = model[unlabeled_tags]

In [41]:
len(unlabeled_tags)

116366

In [42]:
len(article_tag)

140177

In [43]:
sel_train = selfTrainer(topk=1, save_error=True, epoch=150, batch_size=256,
                        verbose=True, keras=False)
sel_train.fit(X_train, y_train, X_test, y_test, unlabeled_vec,X_val, y_val)


Init val error:  0.9281814363712726
Acc at iter 0: 0.9281814363712726 and the dif:  0.0
and acc: 0.9298908480268682 , and f1: 0.9593772804670397, and n new data: 1


KeyboardInterrupt: 

In [16]:
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec


class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''
    
    def __init__(self, path_prefix):
        self.path_prefix = path_prefix
        self.epoch = 0

    def on_epoch_end(self, model):
        output_path = get_tmpfile('{}_epoch{}.model'.format(self.path_prefix, self.epoch))
        model.save(output_path)
        self.epoch += 1
        
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    
    def __init__(self):
        self.epoch = 0
        
    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
    
    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [17]:
model = gensim.models.Word2Vec.load("assets/doc2vecModel/more_school_1000")
doc_vec = model.docvecs.vectors_docs
doc_tag = list(model.docvecs.doctags.keys())
word_vectors = model.wv.vectors

/home/ekvall/anaconda3/envs/kth-cluster/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
article_tag = [t for t in doc_tag if not t[0].islower()]
all_vec = model[article_tag]
tags =np.asarray(article_tag)

In [ ]:
tags

In [52]:
x_un, y_un = (np.load("assets/SemiSupArray/sampleX_iter42000_val0.934_test0.942.npy"),
np.load("assets/SemiSupArray/sampley_iter42000_val0.934_test0.942.npy"))
X_train_c, y_train_c = np.concatenate((X_train, x_un, X_val, X_test)), np.concatenate((y_train, y_un.ravel(), y_val, y_test))

In [54]:
b = GaussianNB()
b.fit(X_train_c, y_train_c)

test_pred = b.predict(unlabeled_vec)
#print(accuracy_score(test_pred, y_test))

In [55]:
test_pred.shape

(116366,)

In [56]:
test_pred.sum()

90135

In [59]:
new_ls = np.array(unlabeled_tags)[test_pred == 0]

In [61]:
uppsala_ls = list()
SU_ls = list()
sodertorn_ls = list() 
for ls in new_ls:
    inst = ls.split("_")[1]
    if inst == "Uppsala":
        uppsala_ls.append(ls)
    elif inst == "SU":
        SU_ls.append(ls)
    elif inst == "sodertorn":
        sodertorn_ls.append(ls)


In [63]:
len(sodertorn_ls)

370

In [131]:
len(SU_ls)

6856

In [155]:
len(uppsala_ls)

19005

In [68]:
all_auth.head()

,Abstracts,Auth_id,Doc_id,Auth_name
0,A model for the enhanced water dissociation th...,u16fm297:u13dx9f6,65923903,"Dahlkild, Anders A.:Behm, Marten"
1,Wood from white spruce Picea glauca that had ...,u16k1pmb:u1lkf75c,40520967,"Zhang, Liming:Henriksson, Gunnar"
2,"We consider the moduli space H-g,H-n of n-poin...",u1mv0zlg,41290309,"Bergstrom, Jonas"
3,In this paper a day-ahead planning algorithm f...,u16u3erw:u1naf2f4:u1fjok0u,40129874,"Matevosyan, Julija:Olsson, Magnus:Soder, Lennart"
4,"In this letter, we develop a fixed-point arith...",u1s42xk3:u12s8cr8,61385618,"Johansson, Christopher:Lansner, Anders B."


In [149]:
def get_raw_name(doc_id):
    auth_name = list()
    for did in doc_id:
        authors = all_auth[all_auth.Doc_id == did].Auth_name.values
        auth_name += list(authors)
    return auth_name
def get_names(raw_name):
    all_name = list()
    for a in raw_name:
        all_name += [auth for auth in a.split(":")]
    return list(set(all_name))
def get_user_id(names, name_t_id):
    Ids = list()
    for name in names:
        Ids.append(name_t_id[str(name)])
    return Ids
def saveAsDf(unique_name, Ids, name):
    data = {"name":unique_name, "id":Ids}
    df = pd.DataFrame(data)
    df.to_csv("assets/" + str(name))

In [143]:
id_to_auth_uppsala = pickle_o.load("./assets/dictionaries/id_to_auth_uppsala")
id_to_auth_su = pickle_o.load("./assets/dictionaries/id_to_auth_SU")

In [151]:
auth_to_id_uppsala = {v:k for v,k in zip(list(id_to_auth_uppsala.values()), list(id_to_auth_uppsala.keys()))}
auth_to_id_su = {v:k for v,k in zip(list(id_to_auth_su.values()), list(id_to_auth_su.keys()))}

In [154]:
#raw_uppsala_name = get_raw_name(uppsala_ls)
#unique_name = get_names(raw_uppsala_name)
#uppsal_id = get_user_id(unique_name, auth_to_id_uppsala)

#raw_su_name = get_raw_name(SU_ls)
#unique_name_su = get_names(raw_su_name)
#su_id = get_user_id(unique_name_su, auth_to_id_su)
#saveAsDf(unique_name_su, su_id, "suNameId")

In [123]:
def saveAsDf(unique_name, Ids, name)
    data = {"name":unique_name, "id":Ids}
    df = pd.DataFrame(data)
    df.to_csv("assets/" + str(name))

In [125]:
uppsala_data = pd.DataFrame(data)
uppsala_data.to_csv("assets/" + str(name))

In [127]:
!ls "assets"

SemiSupArray	 doc2vecModel
assignmentStats  finalproduct
dataframes	 terran-fc671-firebase-adminsdk-k8xbh-0d575a0a99.json
dictionaries	 words.json
divaScrape	 words.txt
diva_2004


In [128]:
uppsala_data.to_csv("assets/uppsalaNameId")

In [99]:
pickle_o.save(unique_name,"uppsala_names")

In [96]:
unique_name[65]

'Simard, Edgar P.'

In [156]:
uppsala_data.shape

(105710, 2)

In [ ]:
num_cluster=250
BGMM = BayesianGaussianMixture(num_cluster, verbose=1)
labels = BGMM.fit_predict(all_vec)


In [ ]:
ls_d= np.array(list(set(ls_d))).astype(int)

In [ ]:
ls_d

In [ ]:
df.head()

In [ ]:
list_of_dict = list()
for a, d in zip(df.Author.values, df.Doc_id.values):
    new_d = dict()
    new_d["name"] = str(a)
    
    abstracts = list()
    all_d = d.split(":")
    new_d["docid"] = all_d
    for ad in all_d:
        abst = df_abs[df_abs.Doc_id == int(ad)].Abstracts.values[0]
        abstracts.append(abst)
    new_d["abstracts"] = abstracts
    list_of_dict.append(new_d)
    

In [ ]:
list_of_dict[0]

In [ ]:
with open('assets/finalproduct/auth_to_abs.json', 'w') as fp:
    json.dump(list_of_dict, fp)

In [ ]:
y = json.dumps(auth_to_abs)

# the result is a Python dictionary:
print(y)

In [ ]:
author = df.Author.values
list_of_author= list()
for i, a in enumerate(author):
    a_dict = dict()
    a_dict["id"]= i
    a_dict["name"]= a
    list_of_author.append(a_dict)


In [ ]:
with open('assets/finalproduct/list_of_author.json', 'w') as fp:
    json.dump(list_of_author, fp)

In [ ]:
nan_ix = [isinstance(i,float) for i in df.Department.values]
df.Department[nan_ix] = "NaN"
department = list(set(df.Department.values))

In [ ]:
department = [make_name_noAscii(d) for d in department]

In [ ]:
department_to_auth= list()
for i, d in enumerate(department):
    author = list(df[df.Department == d].Author.values)
    a_dict = dict()
    a_dict["department"]= d
    a_dict["name"]= author
    department_to_auth.append(a_dict)

In [ ]:
with open('assets/finalproduct/department_to_auth.json', 'w') as fp:
    json.dump(department_to_auth, fp)

In [ ]:
dep_list = list()
for i, d in enumerate(department):
    a_dict = dict()
    a_dict["id"]= i
    a_dict["department"]= d
    dep_list.append(a_dict)

In [ ]:
with open('assets/finalproduct/departments.json', 'w') as fp:
    json.dump(dep_list, fp)

In [ ]:
kth_school_s = pd.Series(np.array(df.Department)).value_counts().sort_values(ascending=False)
plt.figure(figsize=(35,23))
ax = sns.barplot(kth_school_s.index,kth_school_s.values)

ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right",fontsize=30)
ax.set_title("KTH authors distribution(departments)",fontsize=50)

ax.set_ylabel("Counts",fontsize=30)
sns.set(font_scale=3)
plt.gcf().subplots_adjust(bottom=0.40)
#plt.show()
plt.savefig("assets/figures/articleDepartmentFinal")

In [ ]:
len(kth_school_s.index)

In [ ]:
39 - 5

In [ ]:
kth_school_s.values.sum()

In [ ]:
1744 - 884